# OpenStreetMap Data Case Study
#### Author: Li Tang
#### Date: October 8, 2017

### 1. Map Area
#### Boston, Massachusetts, USA
I am using the OpenStreet data of Boston Massachusetts area from mapzen.com https://mapzen.com/data/metro-extracts/metro/boston_massachusetts/. I would like to take this oppotunity to explore Bonston area. The data format is XML. 

### 2. Identifying Problems in the Map

#### The output from audit.py script will be analyzed to visualize any unsual street names and postal codes. 

### 3. Some Problems Encountered in the Map

1. Some abbreviated street types, for example, 'St.', 'St', 'st', 'ST', and 'ST'. 
2. A few inconsistent postal codes, like 'MA 02118' and '02118-0239'.

#### 3.1 Correct inconsistent street names:

This part of the project is to fix the street types as below: 

          {
           "St": "Street",
           "st": "Street",
           "Street.": "Street", 
           "street": "Street",
           "St.": "Street",
           "St,": "Street", 
           "ST": "Street",
           "Rd.": "Road", 
           "Ave": "Avenue", 
           "Ave.": "Avenue",
           "Pkwy": "Parkway", 
           "rd.": "Road", 
           "Ct": "Court",  
           "Dr": "Drive", 
           "Rd": "Road", 
           "Hwy": "Highway", 
           "Sq.": "Square"
           }

The procedure in detail is demonstrated in audit_street.py.

Briefly stated: 
1. Create a list of expected street types that do not need to be cleaned.
2. The function "audit_street_type" collects the last words in the "street_name" strings, if they are not within the expected list, then stores them in the dictionary "street_types". This will allow me to see the nonuniform and abbreviated street types being used and give me a better sense of the data in the big picture.
3. The "is_street_name" function looks for tags that specify street names (k="addr:street").
4. The "audit" function returns a dictionary that match the above function conditions.
5. The update_name function takes an old name to mapping dictionary, and update to a new one.

#### 3.2 Correct postal codes:

1. Boston area postal codes begin with "021,022, and 024". Some of their formats are 5 digits, and some are 9 digits, and some even begin with state characters. 
2. The leading and trailing characters other than the main 5 digits will be dropped. After cleaning, 'MA 02118' will be fixed to '02118' and '02118-0239' will become '02118'. The detailed procedure is illustrated in audit_postcode.py. 

### 4. CSV File Generation for SQL Database Preparation

The elements in the OSM XML file will be parsed, transforming the document data into the tabular format. The final purpose is to prepare a .csv files that are easy to be imported to a SQL database. The detailed procedure is shown in data_process.py. 

### 5. SQL DataBase and Tables

#### 5.1 Create SQL Database and Tables

In [1]:
import sqlite3
import csv
from pprint import pprint

sqlite_file = "project.db"
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS nodes')
cur.execute('''
   Create Table nodes(id INTEGER, lat REAL, lon REAL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT)
''')
conn.commit()

cur.execute('DROP TABLE IF EXISTS ways')
cur.execute('''
   Create Table ways(id INTEGER, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT)
''')
conn.commit()

cur.execute('DROP TABLE IF EXISTS nodes_tags')
cur.execute('''
   Create Table nodes_tags(id INTEGER, key TEXT, value TEXT, type TEXT)
''')
conn.commit()

cur.execute('DROP TABLE IF EXISTS ways_tags')
cur.execute('''
   Create Table ways_tags(id INTEGER, key TEXT, value TEXT, type TEXT)
''')
conn.commit()

cur.execute('DROP TABLE IF EXISTS ways_nodes')
cur.execute('''
   Create Table ways_nodes(id INTEGER, node_id INTEGER, position INTEGER)
''')
conn.commit()

#### 5.2 Insert Data from CSV Files into SQL Tables

In [2]:
# read in data
with open('nodes.csv', 'rb') as fin_nodes:
    dr = csv.DictReader(fin_nodes) 
    to_db = [(i['id'], i['lat'], i['lon'], i['user'].decode('utf8'), i['uid'],
             i['version'], i['changeset'], i['timestamp']) for i in dr]

# insert data to table
cur.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version,changeset,timestamp) VALUES(?, ?, ?, ?, ?, ?, ?, ?);", to_db)

In [3]:
# read in data
with open('ways.csv', 'rb') as fin_ways:
    dr = csv.DictReader(fin_ways) 
    to_db = [(i['id'], i['user'].decode('utf8'), i['uid'],
             i['version'], i['changeset'], i['timestamp']) for i in dr]

# insert data to table
cur.executemany("INSERT INTO ways(id,user, uid, version,changeset,timestamp) VALUES(?, ?, ?, ?, ?, ?);", to_db)

In [4]:
# read in data
with open('nodes_tags.csv', 'rb') as fin_nodes_tags:
    dr = csv.DictReader(fin_nodes_tags) 
    to_db = [(i['id'], i['key'].decode('utf8'), i['value'].decode('utf8'),
             i['type'].decode('utf8')) for i in dr]

# insert data to table
cur.executemany("INSERT INTO nodes_tags(id,key, value, type) VALUES(?, ?, ?, ?);", to_db)

In [8]:
# read in data
with open('ways_tags.csv', 'rb') as fin_ways_tags:
    dr = csv.DictReader(fin_ways_tags) 
    to_db = [(i['id'], i['key'].decode('utf8'), i['value'].decode('utf8'),
             i['type'].decode('utf8')) for i in dr]
# insert data to table
cur.executemany("INSERT INTO ways_tags(id, key, value,type) VALUES(?, ?, ?, ?);", to_db)

In [10]:
# read in data
with open('ways_nodes.csv', 'rb') as fin_ways_nodes:
    dr = csv.DictReader(fin_ways_nodes) 
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

# insert data to table
cur.executemany("INSERT INTO ways_nodes(id,node_id,position) VALUES(?, ?, ?);", to_db)

### 6. Data Overview

#### 6.1 Summary statistics of the dataset

In [1]:
import os
print('The boston.osm file is {} MB'.format(os.path.getsize('boston.osm')/1.0e6))
print('The project.db file is {} MB'.format(os.path.getsize('project.db')/1.0e6))
print('The nodes.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The nodes_tags.csv file is {} MB'.format(os.path.getsize('nodes_tags.csv')/1.0e6))
print('The ways.csv file is {} MB'.format(os.path.getsize('ways.csv')/1.0e6))
print('The ways_tags.csv is {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e6))
print('The ways_nodes.csv is {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e6)) # Convert from bytes to MB

The boston.osm file is 438.811272 MB
The project.db file is 255.932416 MB
The nodes.csv file is 160.95605 MB
The nodes_tags.csv file is 17.527426 MB
The ways.csv file is 21.045341 MB
The ways_tags.csv is 23.00255 MB
The ways_nodes.csv is 55.58002 MB


#### 6.2 Data Inquiry

#### Number of nodes

In [11]:
cur.execute("SELECT COUNT(*) FROM nodes;")
print(cur.fetchall())


[(1952878,)]


#### Number of ways

In [12]:
cur.execute("SELECT COUNT(*) FROM ways;")
print(cur.fetchall())

[(311076,)]


#### Number of unique users

In [13]:
cur.execute("SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;")
print(cur.fetchall())

[(1421,)]


#### Top 10 contributing users

In [14]:
import pprint
cur.execute("SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC LIMIT 10;")
pprint.pprint(cur.fetchall())

[(u'crschmidt', 1195006),
 (u'jremillard-massgis', 428633),
 (u'wambag', 111175),
 (u'OceanVortex', 90950),
 (u'morganwahl', 67049),
 (u'ryebread', 65963),
 (u'MassGIS Import', 58553),
 (u'ingalls_imports', 32453),
 (u'Ahlzen', 28321),
 (u'mapper999', 14697)]


#### Number of Starbucks

In [16]:
cur.execute("SELECT COUNT(*) FROM nodes_tags WHERE value LIKE '%Starbucks%';")
print(cur.fetchall())

[(64,)]


#### Count Tourism Related Categories Descending

In [18]:
import pprint
cur.execute ("SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL \
             SELECT * FROM ways_tags) tags \
             WHERE tags.key LIKE '%tourism'\
             GROUP BY tags.value \
             ORDER BY count DESC;")
pprint.pprint(cur.fetchall())

[(u'hotel', 103),
 (u'museum', 60),
 (u'artwork', 58),
 (u'attraction', 38),
 (u'viewpoint', 33),
 (u'information', 27),
 (u'picnic_site', 26),
 (u'guest_house', 9),
 (u'hostel', 5),
 (u'motel', 3),
 (u'aquarium', 2),
 (u'chalet', 2),
 (u'apartment', 1),
 (u'gallery', 1),
 (u'theme_park', 1),
 (u'zoo', 1)]


#### 6.3 Further Data Exploration

#### Number of Restaurants in each city descending

In [19]:
import pprint
cur.execute("SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) \
            FROM nodes_tags WHERE value = 'restaurant') i ON nodes_tags.id = i.id WHERE nodes_tags.key = 'city'\
            GROUP BY nodes_tags.value ORDER BY num DESC;")

pprint.pprint(cur.fetchall())

[(u'Boston', 55),
 (u'Cambridge', 51),
 (u'Somerville', 19),
 (u'Brookline', 10),
 (u'Allston', 5),
 (u'East Boston', 5),
 (u'Brookline, MA', 4),
 (u'Chelsea', 4),
 (u'Arlington', 3),
 (u'Medford', 3),
 (u'Watertown', 3),
 (u'Arlington. MA', 2),
 (u'Boston, MA', 2),
 (u'Brighton', 2),
 (u'Charlestown', 2),
 (u'Chestnut Hill', 2),
 (u'Jamaica Plain', 2),
 (u'Malden', 2),
 (u'2067 Massachusetts Avenue', 1),
 (u'Arlington, MA', 1),
 (u'Cambridge, MA', 1),
 (u'Cambridge, Massachusetts', 1),
 (u'Everett', 1),
 (u'Quincy', 1),
 (u'Watertown, MA', 1),
 (u'boston', 1),
 (u'somerville', 1),
 (u'winthrop', 1)]


#### Top 5 Most Popular Fast Food Chain

In [20]:
cur.execute ("SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) \
             FROM nodes_tags WHERE value='fast_food') i ON nodes_tags.id=i.id WHERE nodes_tags.key='name' \
             GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 5;")

pprint.pprint(cur.fetchall())

[(u"Dunkin' Donuts", 13),
 (u'Subway', 12),
 (u"McDonald's", 9),
 (u'Burger King', 8),
 (u"Wendy's", 5)]


#### Top 5 Cafe Chain

In [21]:
cur.execute ("SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) \
             FROM nodes_tags WHERE value='cafe') i ON nodes_tags.id=i.id WHERE nodes_tags.key='name' \
             GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 5;")

pprint.pprint(cur.fetchall())

[(u'Starbucks', 48),
 (u"Dunkin' Donuts", 45),
 (u'Au Bon Pain', 7),
 (u'Dunkin Donuts', 6),
 (u"Peet's Coffee", 5)]


### 8. Conclusions

From the above review, there are only 103 hotels in Boston. A quick search on hotel.com gives me a result of 597 results. OpenStreetMap should encourage hotels, restaurants and other local business owners to publish on OSM, and edit their own content to make it more accurate and make it available for everyone to benefit from.Parks, schools, hotels, restaurants and other local businesses should be able to update and edit their own data on OSM, to ensure the accuracy and completeness. There are potential to extend OpenStreetMap to include travel, walking and biking tour information, user reviews of establishments, local public transit etc. If more people use it, more people adding data, and this makes the maps better for everyone. This could expand OpenStreetMap's user base, and become more consumer focused.

Givin that there're hundreds of contributors for this map, we could expect a great numbers of human errors in this project. A srtuctured input form is strongly recommended so that everyone can input the same data format to reduce the error. 